# Continuous Deployment with AWS Lambda

## Imports

In [ ]:
import json
import os
import requests
import sys
sys.path.append("../")

from app_gradio import app
from question_answer.answer import Pipeline

## Build container image

In [ ]:
os.environ["LAMBDA_NAME"] = "admirer-backend"

In [ ]:
%cd ..

In [ ]:
!docker build --no-cache -t $LAMBDA_NAME . --file api_serverless/Dockerfile

In [ ]:
%cd backend_setup

## Upload to the container registry

In [ ]:
aws_account_id, = !aws sts get-caller-identity \
  --query "Account"
aws_region, = !aws configure get region 

os.environ["AWS_REGION"] = aws_region
os.environ["AWS_ACCOUNT_ID"] = aws_account_id.strip('"')

!echo $AWS_ACCOUNT_ID
!echo $AWS_REGION

In [ ]:
os.environ["ECR_URI"] = ".".join(
    [os.environ["AWS_ACCOUNT_ID"], "dkr", "ecr", os.environ["AWS_REGION"], "amazonaws.com"])

!echo $ECR_URI

In [ ]:
!aws ecr get-login-password --region $AWS_REGION \
  | docker login --username AWS --password-stdin $ECR_URI

In [ ]:
!aws ecr create-repository \
  --repository-name $LAMBDA_NAME \
  --image-scanning-configuration scanOnPush=true --image-tag-mutability MUTABLE \
  | jq -C

In [ ]:
os.environ["IMAGE_URI"] = "/".join([os.environ["ECR_URI"], os.environ["LAMBDA_NAME"]])

In [ ]:
!docker tag $LAMBDA_NAME\:latest $IMAGE_URI\:latest

In [ ]:
!docker push $IMAGE_URI\:latest